> 在非零和矩阵博弈中，二局中人并不是完全对立的，局中人Ⅰ的所得不再是局中人Ⅱ的所失，这种博弈通常记为 $G=\{S_1,S_2;\boldsymbol A, \boldsymbol B\}$，其中 $\boldsymbol A=(a_{ij})_{m\times n},\ \boldsymbol B=(b_{ij})_{m\times n}$，分别是局中人Ⅰ和Ⅱ的赢得矩阵，故称为二人有限非零和博弈，或***双矩阵博弈***

---
### 16.4.1 非合作的双矩阵博弈的纯策略解